In [3]:
val_splits = ["90","91","92","93","94"]
val_sentences = 40000

In [10]:
val_written = []
val_spoken = []
spoken_sen = []
written_sen = []
for idx in val_splits:
    print(f"reading {idx}")
    with open(f"val/output-000{idx}-of-00100", "r") as file:
        ITN_flag = False
        skip = False
        for line in file:
            fields = line.strip().split("\t")
            if not fields[1].isascii():
                print(fields[1])
                skip = True

            if len(fields)==2:
                spoken_sen = " ".join(spoken_sen)
                written_sen = " ".join(written_sen)
                if not skip and ITN_flag and len(written_sen) < 1024 and len(spoken_sen) < 1024:
                    ITN_flag = False
                    val_written.append(written_sen)
                    val_spoken.append(spoken_sen)
                    if len(val_written) == val_sentences:
                        break
                skip = False
                spoken_sen = []
                written_sen = []

            elif len(fields)==3:
                if fields[0] == "ELECTRONIC":
                    skip = True
                elif fields[0] != "PLAIN" and fields[0] != "PUNCT":
                    ITN_flag = True
                    if fields[2] == "<self>":
                        spoken_sen.append(fields[1].casefold())
                    else:
                        spoken_sen.append(fields[2].casefold())
                elif fields[0] == "PUNCT":
                    if fields[1] in '()"':
                        continue
                else:
                    spoken_sen.append(fields[1].casefold())
                written_sen.append(fields[1])
    if len(val_written) == val_sentences:
        break

with open("sampled_val_written.txt", "w") as w_file, open("sampled_val_spoken.txt", "w") as s_file:
    for w in val_written:
        w_file.write(w+"\n")
    for s in val_spoken:
        s_file.write(s+"\n")

reading 90
—
Coupé
Π
α
λ
α
ι
ο
π
ο
λ
η
—
ǀ
ʼ
série
£100
—
José
—
—
—
México
André
—
—
ʻ
Ī
ʻ
년
주
차
Σ
τ
η
ν
π
λ
α
τ
ε
ι
α
π
α
λ
ι
ο
ι
Α
γ
α
ν
α
κ
τ
ι
σ
μ
ε
ν
ο
ι
—
—
214.1/km²
567.0/mi²
663.0/km²
—
142.2/km²
173.0/km²
—
—
—
—
£100,000
Maboké
—
—
—
—
—
—
牧
瀬
紅
莉
栖
—
encadré
compétition
—
ợ
Inés
Inés
Pérez
—
/km²
—
1/km²
Métis
—
—
Lévy
—
née
—
—
£13.15
«
—
—
—
Monné
27.5/km²
佛
手
巖
棺
材
石
靈
龜
上
山
僧
人
石
—
—
—
—
—
—
Chiloé
—
—
—
一
進
會
일
진
회
—
Pétain
а
а
е
о
М
а
а
В
е
а
—
Bézout
—
—
Lycée
舟
小
屋
風
土
と
か
た
ち
—
—
Données
κ
é
—
—
189.7/km²
21.5/km²
Méjanelle
—
ε
orcné
orcnéas
9 km²
渋
谷
ホ
ー
ル
Fédération
—
—
Pérez
Pérez
ố
ữ
き
ら
き
ら
キ
ラ
ー
«
О
в
н
е
с
е
н
и
и
и
з
м
е
н
е
н
и
й
в
о
т
д
е
л
ь
н
ы
е
З
а
к
о
н
ы
М
а
г
а
д
а
н
с
к
о
й
о
б
л
а
с
т
и
»
Esmé
Esmé
宮
内
勝
典
—
—
per km²
徐
冠
林
黄
章
サ
ー
キ
ッ
ト
の
狼
コ
ミ
ッ
ク
ス
コ
ミ
ッ
ク
Détente's
—
Σ
ε
λ
ι
ν
ο
υ
ν
τ
α
ς
—
volée
Л
е
в
Г
у
д
к
о
в
с
т
а
л
л
а
у
р
е
а
т
о
м
п
р
е
м
и
и
П
о
л
и
т
П
р
о
с
в
е
т
с
е
з
о
н
а
β
β
—
—
—
—
—
—
—
Général
—
—
—
ˌ
ɪ
ˈ
ɪ
—
—
—
—
—
€39
€

In [2]:
from tqdm import tqdm

In [11]:
start_idx = 0
end_idx = 89
counter = {}
contained_classes = set()
max_num_each_class = 100000 # maximum number of instances for each classes
excluded_classes = set()
excluded_classes.add("ELECTRONIC")
finished = False
for idx in range(start_idx, end_idx+1):
    print(f"reading {idx}")
    train_written = []
    train_spoken = []
    spoken_sen = []
    written_sen = []
    with open("train/output-000{:0>2d}-of-00100".format(idx), "r") as file:
        ITN_flag = False
        skip = False
        for line in file:
            fields = line.strip().split("\t")
            # if not fields[1].isascii():
            #     print(fields[1])
            #     skip = True

            if len(fields)==2:
                spoken_sen = " ".join(spoken_sen)
                written_sen = " ".join(written_sen)
                if not skip and ITN_flag and len(written_sen) < 1024 and len(spoken_sen) < 1024:
                    ITN_flag = False
                    train_written.append(written_sen)
                    train_spoken.append(spoken_sen)
                    for cls in contained_classes:
                        counter[cls] = counter.get(cls, 0) + 1
                        if cls not in excluded_classes and counter[cls] >= max_num_each_class:
                            excluded_classes.add(cls)
                            print(cls)
                            if len(excluded_classes) >= 11:
                                finished = True
                skip = False
                if finished:
                    break
                contained_classes = set()
                spoken_sen = []
                written_sen = []

            elif len(fields)==3:
                if fields[0] in excluded_classes:
                    skip = True
                elif fields[0] != "PLAIN" and fields[0] != "PUNCT" and fields[0] != "ELECTRONIC":
                    skip = False
                    ITN_flag = True
                    contained_classes.add(fields[0])
                    if fields[2] == "<self>":
                        spoken_sen.append(fields[1].casefold())
                    else:
                        spoken_sen.append(fields[2].casefold())
                elif fields[0] == "PUNCT":
                    if fields[1] in '()"':
                        continue
                else:
                    spoken_sen.append(fields[1].casefold())
                written_sen.append(fields[1])
    with open("sampled_train_written.txt", "a") as w_file, open("sampled_train_spoken.txt", "a") as s_file:
        for w in train_written:
            w_file.write(w+"\n")
        for s in train_spoken:
            s_file.write(s+"\n")
    if finished:
        break

reading 0
DATE
LETTERS
reading 1
CARDINAL
reading 2
reading 3
VERBATIM
reading 4
reading 5
reading 6
reading 7
reading 8
reading 9
reading 10
MEASURE
reading 11
ORDINAL
reading 12
reading 13
reading 14
reading 15
reading 16
reading 17
DECIMAL
reading 18
reading 19
reading 20
reading 21
reading 22
reading 23
MONEY
reading 24
reading 25
reading 26
reading 27
reading 28
reading 29
reading 30
reading 31
reading 32
reading 33
reading 34
reading 35
reading 36
reading 37
reading 38
reading 39
reading 40
reading 41
reading 42
reading 43
reading 44
reading 45
reading 46
reading 47
reading 48
TELEPHONE
reading 49
reading 50
reading 51
reading 52
reading 53
reading 54
reading 55
reading 56
reading 57
reading 58
reading 59
reading 60
reading 61
reading 62
reading 63
reading 64
reading 65
DIGIT


In [6]:
counter

{'DATE': 100185,
 'LETTERS': 100185,
 'CARDINAL': 100182,
 'VERBATIM': 100176,
 'DECIMAL': 100139,
 'MEASURE': 100114,
 'MONEY': 100105,
 'ORDINAL': 100082,
 'TIME': 100042,
 'DIGIT': 100000,
 'FRACTION': 99948,
 'TELEPHONE': 99747,
 'ADDRESS': 98185}